In [2]:
import pandas as pd
from hs_gimme.db_facade.db_facade_factory import get_mongo_client_db
from hs_tree_blenders_creation.batch_runners.data_sets import get_account_test_set
from tqdm import tqdm
from collections import Counter
import random as rn
from datetime import datetime

In [3]:
#ACCOUNTS = ['essex', 'detroit', 'jamaica', 'paris', 'dallas', 'colorado', 'jaipur', 'cairo', 'nashville', 'exeter']
ACCOUNTS = ['atlanta']
ENV = 'production'
REQS_MIN_DATE = datetime(2024, 3, 1)
REQS_MAX_DATE = datetime(2024, 5, 1)
REQ_STATUSES = ['Filled', 'Open']
COUNTRIES = ['United States', 'US', 'UNITED STATES']

In [4]:
countries = Counter()
for account in tqdm(['detroit']):
    mongo = get_mongo_client_db(ENV, account)
    countries.update([r.get('country') for r in mongo.req.find({}, ['country']).limit(10000)])
countries.most_common(10)

  0%|          | 0/1 [00:00<?, ?it/s]

2024-07-10T12:23:44.828852Z [info     ] Mongos instances selected      environment=local file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/connection_string_builder.py function_name=get_selected_mongos_instances hosts=['applicativedb-prod-mongos-1.omcomcom.com', 'applicativedb-prod-mongos-0.omcomcom.com'] line_number=38 module=hs_gimme.db_facade.connection_string_builder pid=1981


100%|██████████| 1/1 [00:04<00:00,  4.03s/it]


[('United States', 9902), (None, 96), ('United Kingdom', 2)]

In [5]:
raw_data = []

proj = ['ats_application.eeo', 
          'ats_application.integration_data.final_round_score_sent_to_ats',
          'grade_data.explainable_score_data.explain']

for i, account in enumerate(ACCOUNTS):
    mongo = get_mongo_client_db(ENV, account)
    req_ids = [r['_id'] for r in mongo.req.find({'external_status': {'$in': REQ_STATUSES}, 
                                                 'job_create_date': {
                                                                     '$gt': REQS_MIN_DATE,
                                                                     '$lte': REQS_MAX_DATE
                                                                    },
                                                 'country': {'$in': COUNTRIES}}, [])]
    
   # req_ids = rn.sample(req_ids, min(len(req_ids), 5000))
    print('='*20)
    print(f'{i+1}/{len(ACCOUNTS)} {account} {len(req_ids)} reqs')
    print('='*20)
    query = {'req_id': {'$in': req_ids}, 'ats_application.integration_data.final_round_score_sent_to_ats': {'$exists': True}}
    
    for app in mongo.application.find(query, proj):
        raw_data.append(app)
len(raw_data)

1/1 atlanta 762 reqs


88558

In [6]:
data = []
for app in raw_data:
    if 'grade_data' in app and 'explainable_score_data' in app['grade_data']:
        
        grade = app['ats_application']['integration_data']['final_round_score_sent_to_ats']
        if grade in {'A', 'B'}:
            is_priority = 1
        elif grade in {'C', 'D'}:
            is_priority = 0
        else:
            is_priority = None
        
        if 'raw_grade_without_rnd' in app['grade_data']['explainable_score_data']['explain']:
            d = {
                'gender':  app['ats_application'].get('eeo', {}).get('gender', 'Missing'),
                'race': app['ats_application'].get('eeo', {}).get('race', 'Missing'),
                'grade': grade,
                'is_priority': is_priority,
               # 'raw_score': app['grade_data']['explainable_score_data']['explain']['raw_predicted_proba'],
                'raw_final_score': app['grade_data']['explainable_score_data']['explain']['raw_grade_without_rnd'],
                'raw_final_score_priority': app['grade_data']['explainable_score_data']['explain']['raw_grade_without_rnd'] >= 3
            }

            for g in ['A', 'B', 'C', 'D']:
                d[g] = grade == g
            data.append(d)
        
df = pd.DataFrame(data)

In [7]:
df.head()

,gender,race,grade,is_priority,raw_final_score,raw_final_score_priority,A,B,C,D
0,Female,Asian,C,0.0,2.430928,False,False,False,True,False
1,Female,Asian,D,0.0,1.839463,False,False,False,False,True
2,Female,Asian,D,0.0,1.234894,False,False,False,False,True
3,Male,Asian,C,0.0,3.598693,True,False,False,True,False
4,Male,Black or African American,D,0.0,1.225797,False,False,False,False,True


In [8]:
medians = {}
for field in ['raw_final_score']:
    medians[field] = df[field].median()
    
    df[field + '_gt_median'] = df[field] >= df[field].median()
medians

{'raw_final_score': 1.9777640213375336}

In [9]:
df['race'].value_counts(normalize=True, dropna=False)

Black or African American              0.277730
White                                  0.207065
Asian                                  0.205793
Hispanic or Latino                     0.194407
Decline to Identify                    0.050913
Two or more races                      0.045845
Native Hawaiian or Pacific Islander    0.008609
American Indian or Alaska Native       0.006607
Missing                                0.003032
Name: race, dtype: float64

In [10]:
too_small_races = {r for r in df['race'].unique() if df['race'].value_counts()[r] < 0.02*len(df)} - {'Decline to Identify', 'Missing'}
too_small_races

{'American Indian or Alaska Native', 'Native Hawaiian or Pacific Islander'}

In [11]:
df['gender'].value_counts()

Male       42249
Female     41650
Missing     2523
Name: gender, dtype: int64

In [12]:
too_small_genders = {r for r in df['gender'].unique() if df['gender'].value_counts()[r] < 0.02*len(df)} - {'Decline to Identify', 'Missing'}
too_small_genders

set()

In [13]:
df = df[~df['gender'].isin(too_small_genders) & ~df['race'].isin(too_small_races)]

In [14]:
len(df)

85107

In [15]:
df['race_disclose'] = df['race'].apply(lambda x: 'Disclose' if x not in {'Decline to Identify', 'Missing'} else x)
df['gender_disclose'] = df['gender'].apply(lambda x: 'Disclose' if x not in {'Decline to Identify', 'Missing'} else x)

In [16]:
gb_dfs = []
for gb_fields in [['gender'], ['race'], ['gender', 'race']]:
    
    impact_df = df
    for f in gb_fields:
        impact_df = impact_df[impact_df[f + '_disclose'] == 'Disclose']
        
    gb = impact_df.groupby(gb_fields).agg(
        count=('gender', 'count'),
        # raw_score_scoring_rate=('raw_score_gt_median', 'mean'),
        # final_score_scoring_rate=('final_score_gt_median', 'mean'),
        selection_rate=('is_priority', 'mean'),
        # A_selection_rate=('A', 'mean'),
        # B_selection_rate=('B', 'mean'),
        # C_selection_rate=('C', 'mean'),
        raw_selection_rate=('raw_final_score_priority', 'mean')
        # D_selection_rate=('D', 'mean')
    ) 
    
    # gb = gb[gb['count'] > 0.02*gb['count'].sum()
    # for f in ['raw_score_scoring_rate', 'final_score_scoring_rate', 'is_priority_selection_rate', 
    #           'A_selection_rate', 'B_selection_rate', 'C_selection_rate', 'D_selection_rate']:
    for f in ['selection_rate', 'raw_selection_rate',]:
    
        gb[f + '_impact_ratio'] = gb[f] / gb[f].max()
    
    gb_dfs.append(gb)
    
                                
ir = pd.concat(gb_dfs)

In [17]:
writer = pd.ExcelWriter(f'~/Downloads/general_audit_report_2024_02_12.xlsx', engine="xlsxwriter")

In [22]:
ir

,count,selection_rate,raw_selection_rate,selection_rate_impact_ratio,raw_selection_rate_impact_ratio
Female,40912,0.713092,0.322961,1.000000,0.987789
Male,41697,0.683646,0.326954,0.958708,1.000000
Asian,17785,0.584931,0.406410,0.763806,1.000000
Black or African American,24002,0.765811,0.297767,1.000000,0.732676
Hispanic or Latino,16801,0.727397,0.288376,0.949839,0.709569
Two or more races,3962,0.744321,0.297325,0.971938,0.731588
White,17895,0.674714,0.310198,0.881044,0.763265
"(Female, Asian)",6846,0.589103,0.400526,0.760224,0.974968
"(Female, Black or African American)",13510,0.774907,0.309400,1.000000,0.753149
"(Female, Hispanic or Latino)",8379,0.725743,0.290488,0.936554,0.707112


In [18]:
report = ir[['count', 'raw_selection_rate', 'raw_selection_rate_impact_ratio']]

In [19]:
report.columns = ['Count', 'Selection Rate', 'Impact Ratio']

In [20]:
report

,Count,Selection Rate,Impact Ratio
Female,40912,0.322961,0.987789
Male,41697,0.326954,1.000000
Asian,17785,0.406410,1.000000
Black or African American,24002,0.297767,0.732676
Hispanic or Latino,16801,0.288376,0.709569
Two or more races,3962,0.297325,0.731588
White,17895,0.310198,0.763265
"(Female, Asian)",6846,0.400526,0.974968
"(Female, Black or African American)",13510,0.309400,0.753149
"(Female, Hispanic or Latino)",8379,0.290488,0.707112


In [21]:
summary = pd.Series({
    'Report creation date': datetime.now(),
    'Accounts': ACCOUNTS,
    'Reqs min date': REQS_MIN_DATE,
    'Reqs max date': REQS_MAX_DATE,
    'Reqs Locations': COUNTRIES,
    'Reqs External Status': REQ_STATUSES,
    'Total number of Candidates': len(df),
}).reset_index()


summary.to_excel(writer, sheet_name="Summary", index=False)
summary

,index,0
0,Report creation date,2024-07-10 15:27:40.149614
1,Accounts,[atlanta]
2,Reqs min date,2024-03-01 00:00:00
3,Reqs max date,2024-05-01 00:00:00
4,Reqs Locations,"[United States, US, UNITED STATES]"
5,Reqs External Status,"[Filled, Open]"
6,Total number of Candidates,85107


In [ ]:
report.to_excel(writer, sheet_name="Impact Ratio")

In [ ]:
disclose = pd.DataFrame( {'Race': df['race_disclose'].value_counts(normalize=False), 
               'Gender': df['gender_disclose'].value_counts(normalize=False)})
disclose.to_excel(writer, sheet_name="% Disclose")
disclose

In [ ]:
writer.close()

In [23]:
ir.to_csv('/Users/dima/Downloads/atlanta_audit.csv')